In [1]:
import os
import warnings

import findfile
from pyabsa import ABSAInstruction as absa_instruction

warnings.filterwarnings("ignore")
import pandas as pd

c:\Users\User\Downloads\MDS22_ABSA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-04 22:16:04] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






In [2]:
model_out_path = 'checkpoints/absa/laptop14'

In [3]:
id_train_file_path = '../datasets'
id_test_file_path = '../datasets'

id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "laptop_quad_train.tsv")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "laptop_quad_test.tsv")

../datasets\laptop_quad_train.tsv.jsonl
../datasets\laptop_quad_test.tsv.jsonl


In [4]:
id_tr_df = pd.DataFrame(id_tr_df)
id_te_df = pd.DataFrame(id_te_df)

In [5]:
id_tr_df

,text,labels
0,acer wants $ 170 to just look at it then add t...,"[{'aspect': 'acer', 'opinion': 'NULL', 'polari..."
1,update : i repaired it myself for $ 12 .,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
2,i had nothing to lose since it was a paper wei...,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
3,the shame of it is knowing it took me 15 minut...,"[{'aspect': 'acer', 'opinion': 'NULL', 'polari..."
4,first one that they shipped was obviously defe...,"[{'aspect': 'NULL', 'opinion': 'defective', 'p..."
...,...,...
2929,i think the sound could be better .,"[{'aspect': 'sound', 'opinion': 'could be bett..."
2930,i didn ' t want to spend much on this as it is...,"[{'aspect': 'performs', 'opinion': 'better', '..."
2931,"an awesome product , well built - well worth y...","[{'aspect': 'product', 'opinion': 'awesome', '..."
2932,backlit keyboard is great ; feels sturdy ; fas...,"[{'aspect': 'backlit keyboard', 'opinion': 'gr..."


In [6]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [7]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [8]:
model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)


[2025-03-04 22:16:15] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-03-04 22:16:15] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-03-04 22:16:15] (2.4.1.post1) Checkpoint:kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined is not found, you can raise an issue for requesting shares of checkpoints
[2025-03-04 22:16:15] (2.4.1.post1) No checkpoint found in Model Hub for task: kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined


In [9]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 3075/3075 [00:00<00:00, 16751.29 examples/s]


In [10]:
import torch
print(torch.__version__)  # Should NOT contain '+cpu'
print(torch.version.cuda)  # Should print 12.x or 11.8 (not None)
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should print RTX 4060 Ti

2.5.1+cu121
12.1
True
NVIDIA GeForce RTX 4060 Ti


In [11]:
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 16,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    'fp16': False,
}

model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.249693
2,No log,0.190167
3,No log,0.186589
4,No log,0.190650
5,No log,0.196374
6,No log,0.222241
7,No log,0.242163
8,No log,0.269575
9,No log,0.271502
10,No log,0.303850


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
